In [39]:
import pandas as pd
import os
import sys
import numpy as np

# suppress pandas purely educational warnings
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import pull_csv_from_directory, upload_csv_aws, filter_counties
from scripts.utils.write_metadata import append_metadata

In [2]:
# pull csv from aws
bucket_name = 'ca-climate-index'
aws_dir = '2a_subset/society_economy/economic_health/bureau_labor_statistics/employment_hachman_index/'

pull_csv_from_directory(bucket_name, aws_dir, search_zipped=False)

aws_dir = '1_pull_data/society_economy/economic_health/'
pull_csv_from_directory(bucket_name, aws_dir, search_zipped=False)

Saved DataFrame as 'employment_data_hachman_subset.csv'
Saved DataFrame as 'ACSDT5Y2022.B19083-Column-Metadata.csv'
Saved DataFrame as 'ACSDT5Y2022.B19083-Data.csv'
Saved DataFrame as 'ACSST5Y2022.S1901-Column-Metadata.csv'


c:\Users\jesse\eagle\carb-climate-index-4\scripts\data_metric_calc\../..\scripts\utils\file_helpers.py:100: DtypeWarning: Columns (2,3,34,35,66,67,98,99) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_data)


Saved DataFrame as 'ACSST5Y2022.S1901-Data.csv'


c:\Users\jesse\eagle\carb-climate-index-4\scripts\data_metric_calc\../..\scripts\utils\file_helpers.py:100: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_data)


Saved DataFrame as 'allhlcn22.csv'
Saved DataFrame as 'data_layout.csv'


In [3]:
hachman_data = pd.read_csv('employment_data_hachman_subset.csv')
gini_index_data = pd.read_csv('ACSDT5Y2022.B19083-Data.csv')
median_income_data = pd.read_csv('ACSST5Y2022.S1901-Data.csv')


C:\Users\jesse\AppData\Local\Temp\ipykernel_16892\2131310057.py:3: DtypeWarning: Columns (2,3,34,35,66,67,98,99) have mixed types. Specify dtype option on import or set low_memory=False.
  median_income_data = pd.read_csv('ACSST5Y2022.S1901-Data.csv')


In [4]:
county_tract = "s3://ca-climate-index/0_map_data/ca_tract_county_population_2022.csv"
ca_county_tract = pd.read_csv(county_tract)
ca_county_tract = ca_county_tract.rename(columns={'Census Tract': 'GEO_ID'})
ca_county_tract = ca_county_tract.drop(columns={'Unnamed: 0', 'COUNTYFP', 'County', 'Total Population 2021'})
ca_county_tract

,GEO_ID
0,6085504321
1,6085504410
2,6085507003
3,6085507004
4,6085502204
...,...
9124,6059001303
9125,6059001304
9126,6059001401
9127,6013367200


Gini

In [5]:
gini_index_data_cleaned = gini_index_data.drop(columns={'Unnamed: 4', 'B19083_001M', 'NAME'})
gini_index_data_cleaned = gini_index_data_cleaned.rename(columns={'B19083_001E':'est_gini_index'})
gini_index_data_cleaned = gini_index_data_cleaned[1:]
gini_index_data_cleaned['GEO_ID'] = gini_index_data_cleaned['GEO_ID'].str[10:]

gini_index_data_cleaned

,GEO_ID,est_gini_index
1,6001400100,0.4228
2,6001400200,0.4084
3,6001400300,0.4615
4,6001400400,0.5063
5,6001400500,0.4571
...,...,...
9125,6115040902,0.4252
9126,6115041001,0.5024
9127,6115041002,0.4875
9128,6115041101,0.4072


In [14]:
# Convert the 'GEO_ID' column in ca_county_tract DataFrame to string data type
ca_county_tract['GEO_ID'] = ca_county_tract['GEO_ID'].astype(str)

# Now, you can proceed with merging
gini_merged_data = pd.merge(gini_index_data_cleaned, ca_county_tract, on='GEO_ID', how='right')

# View the merged DataFrame
gini_merged_data

,GEO_ID,est_gini_index
0,6085504321,0.4197
1,6085504410,0.4309
2,6085507003,0.5357
3,6085507004,0.4793
4,6085502204,0.4010
...,...,...
9124,6059001303,0.3590
9125,6059001304,0.3582
9126,6059001401,0.4594
9127,6013367200,0.4130


Median

In [13]:
columns_to_keep = ['GEO_ID', 'S1901_C01_012E']

# Drop all columns except the specified ones
median_income_data_cleaned = median_income_data.drop(columns=[col for col in median_income_data.columns if col not in columns_to_keep])

median_income_data_cleaned = median_income_data_cleaned.rename(columns={'S1901_C01_012E':'est_median_income_dollars'})
median_income_data_cleaned = median_income_data_cleaned[1:]
median_income_data_cleaned['GEO_ID'] = median_income_data_cleaned['GEO_ID'].str[10:]
median_income_data_cleaned

,GEO_ID,est_median_income_dollars
1,6001400100,234236
2,6001400200,225500
3,6001400300,164000
4,6001400400,158836
5,6001400500,95078
...,...,...
9125,6115040902,54265
9126,6115041001,67321
9127,6115041002,102534
9128,6115041101,37018


In [15]:
# Convert the 'GEO_ID' column in ca_county_tract DataFrame to string data type
ca_county_tract['GEO_ID'] = ca_county_tract['GEO_ID'].astype(str)

# Now, you can proceed with merging
median_merged_data = pd.merge(median_income_data_cleaned, ca_county_tract, on='GEO_ID', how='right')

# View the merged DataFrame
median_merged_data

,GEO_ID,est_median_income_dollars
0,6085504321,172857
1,6085504410,143173
2,6085507003,240179
3,6085507004,198306
4,6085502204,67552
...,...,...
9124,6059001303,93363
9125,6059001304,68984
9126,6059001401,76319
9127,6013367200,90941


Hachman

In [21]:
hachman_data

,Area\nCode,St,Cnty,Own,NAICS,Year,Qtr,Area Type,St Name,Area,Ownership,Industry,Annual Average Status Code,Annual Average Establishment Count,Annual Average Employment,Annual Total Wages,Annual Average Weekly Wage,Annual Average Pay,Employment Location Quotient Relative to U.S.,Total Wage Location Quotient Relative to U.S.
0,6000,6,0,0,10,2022,A,State,California,California -- Statewide,Total Covered,"10 Total, all industries",NaN,"1,706,672","17,903,539","1,511,706,499,616","1,624","84,436",1.00,1.00
1,6000,6,0,1,10,2022,A,State,California,California -- Statewide,Federal Government,"10 Total, all industries",NaN,"3,286","247,795","23,655,167,888","1,836","95,463",0.73,0.62
2,6000,6,0,2,10,2022,A,State,California,California -- Statewide,State Government,"10 Total, all industries",NaN,"13,784","492,218","47,749,398,000","1,866","97,009",0.91,1.00
3,6000,6,0,3,10,2022,A,State,California,California -- Statewide,Local Government,"10 Total, all industries",NaN,"19,344","1,724,972","135,804,850,033","1,514","78,729",1.04,1.11
4,6000,6,0,5,10,2022,A,State,California,California -- Statewide,Private,"10 Total, all industries",NaN,"1,670,259","15,438,555","1,304,497,083,695","1,625","84,496",1.01,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,6999,6,999,5,1024,2022,A,County,California,"Unknown Or Undefined, California",Private,1024 Professional and business services,NaN,"26,188","236,341","25,491,212,708","2,074","107,858",3.05,2.03
1070,6999,6,999,5,1025,2022,A,County,California,"Unknown Or Undefined, California",Private,1025 Education and health services,NaN,"4,401","30,314","2,854,407,977","1,811","94,162",0.38,0.34
1071,6999,6,999,5,1026,2022,A,County,California,"Unknown Or Undefined, California",Private,1026 Leisure and hospitality,NaN,"2,121","10,276","932,147,336","1,744","90,713",0.19,0.33
1072,6999,6,999,5,1027,2022,A,County,California,"Unknown Or Undefined, California",Private,1027 Other services,N,"3,031",0,0,0,0,0.00,0.00


In [22]:
# Get unique entries in 'Column1'
unique_entries = hachman_data['Area'].unique()

print(unique_entries)

['California -- Statewide' 'Alameda County, California'
 'Alpine County, California' 'Amador County, California'
 'Butte County, California' 'Calaveras County, California'
 'Colusa County, California' 'Contra Costa County, California'
 'Del Norte County, California' 'El Dorado County, California'
 'Fresno County, California' 'Glenn County, California'
 'Humboldt County, California' 'Imperial County, California'
 'Inyo County, California' 'Kern County, California'
 'Kings County, California' 'Lake County, California'
 'Lassen County, California' 'Los Angeles County, California'
 'Madera County, California' 'Marin County, California'
 'Mariposa County, California' 'Mendocino County, California'
 'Merced County, California' 'Modoc County, California'
 'Mono County, California' 'Monterey County, California'
 'Napa County, California' 'Nevada County, California'
 'Orange County, California' 'Placer County, California'
 'Plumas County, California' 'Riverside County, California'
 'Sacramento 

In [37]:
hachman_data_cleaned = hachman_data[['Area', 'Industry', 'Annual Average Employment']]

# Remove any mention of 'county' within the legalAgencyName column
hachman_data_cleaned['Area'] = hachman_data_cleaned['Area'].str.replace(' -- Statewide', '', case=False)
hachman_data_cleaned['Area'] = hachman_data_cleaned['Area'].str.replace(' County, California', '', case=False)
#hachman_data_cleaned['county'] = hachman_data_cleaned['county'].str.replace(', california', '', case=False)

unique_entries = hachman_data_cleaned['Area'].unique()

C:\Users\jesse\AppData\Local\Temp\ipykernel_16892\3747709064.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hachman_data_cleaned['Area'] = hachman_data_cleaned['Area'].str.replace(' -- Statewide', '', case=False)
C:\Users\jesse\AppData\Local\Temp\ipykernel_16892\3747709064.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hachman_data_cleaned['Area'] = hachman_data_cleaned['Area'].str.replace(' County, California', '', case=False)


In [44]:
filtered_hachman_data, omitted_data = filter_counties(hachman_data_cleaned, 'Area')

print('Counties kept:', len(filtered_hachman_data))
print('Omitted data entries:', len(omitted_data))

Counties kept: 1038
Omitted data entries: 36


In [46]:
omitted_data

,Area,Industry,Annual Average Employment
0,California,"10 Total, all industries","17,903,539"
1,California,"10 Total, all industries","247,795"
2,California,"10 Total, all industries","492,218"
3,California,"10 Total, all industries","1,724,972"
4,California,"10 Total, all industries","15,438,555"
5,California,101 Goods-producing,"2,685,113"
6,California,1011 Natural resources and mining,"436,965"
7,California,1012 Construction,"912,359"
8,California,1013 Manufacturing,"1,335,789"
9,California,102 Service-providing,"12,753,442"


In [ ]:
# Calculate total employment for each county and for California
county_total_employment = hachman_data[hachman_data['Area'] != 'California -- Statewide'].groupby('Area')['Annual Average Employment'].sum()
california_total_employment = hachman_data[hachman_data['Area'] == 'California -- Statewide']['Annual Average Employment'].sum()

county_total_employment = total_employment_df.set_index('Area')['Annual Average Employment']
california_total_employment = total_employment_df[total_employment_df['Area'] == 'California Total']['Annual Average Employment'].values[0]


# Calculate shares
hachman_data['EMPCOUNTYj'] = hachman_data.apply(lambda row: row['Annual Average Employment'] / county_total_employment[row['Area']] 
                            if row['Area'] != 'California -- Statewide' else 0, axis=1)
hachman_data['EMPCAj'] = hachman_data.apply(lambda row: row['Annual Average Employment'] / california_total_employment 
                        if row['Area'] == 'California -- Statewide' else 0, axis=1)

# Create a dictionary to hold EMPCAj for each industry
EMPCAj_dict = hachman_data[hachman_data['Area'] == 'California Total'].set_index('Industry')['EMPCAj'].to_dict()

# Assign EMPCAj to each county-industry pair
hachman_data['EMPCAj'] = hachman_data['Industry'].map(EMPCAj_dict)

# Compute the Hachman Index for each county
hachman_indices = {}
for county in county_total_employment.index:
    county_df = hachman_data[hachman_data['Area'] == county]
    HI = 1 / (sum((county_df['EMPCOUNTYj'] / county_df['EMPCAj']) * county_df['EMPCOUNTYj']))
    hachman_indices[county] = HI

# Display the Hachman Index for each county
print(hachman_indices)